<a href="https://colab.research.google.com/github/07423314796/NLPCW1/blob/main/NLPCW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
#import dataset:
Data=pd.read_csv(r"/olid-training-v1.0.tsv", index_col = False,sep="\t")


In [3]:
New_data=pd.DataFrame(Data)

In [4]:
#Subtask_a groups: Output: array(['OFF', 'NOT'], dtype=object)
New_data['subtask_a'].unique()

array([nan, 'IND', 'OTH', 'GRP'], dtype=object)

In [5]:
#subtask_b groups: output: array(['UNT', 'TIN', nan], dtype=object)
New_data['subtask_b'].unique()

array(['UNT', 'TIN', nan], dtype=object)

In [6]:
#Subtask_c Group: output:array([nan, 'IND', 'OTH', 'GRP'], dtype=object)
New_data['subtask_c'].unique()

array([nan, 'IND', 'OTH', 'GRP'], dtype=object)

In [7]:
#importing libraries:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

In [8]:
#Preprocessing :
import re
from sklearn.base import BaseEstimator, TransformerMixin
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk

nltk.download('stopwords')
nltk.download('punkt')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [9]:
#cleaning the text data
class TextPreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.stop_words = set(stopwords.words('english'))

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X.apply(self._preprocess)

    def _preprocess(self, text):
        # Remove non-alphabetic characters
        text = re.sub(r'[^a-zA-Z\s]', '', text)
        # Tokenize and remove stop words
        tokens = word_tokenize(text)
        tokens = [word for word in tokens if word.lower() not in self.stop_words]
        # Join tokens back into a string
        return ' '.join(tokens)


In [10]:
# Apply text preprocessing
New_data['clean_text'] = TextPreprocessor().fit_transform(New_data['tweet'])


In [11]:
# Define features and labels
X = New_data['clean_text']
y_a = New_data['subtask_a']
y_b = New_data['subtask_b'].fillna('NONE')
y_c = New_data['subtask_c'].fillna('NONE')


In [12]:
# Split the data
X_train_a, X_test_a, y_train_a, y_test_a = train_test_split(X, y_a, test_size=0.2, random_state=42)
X_train_b, X_test_b, y_train_b, y_test_b = train_test_split(X, y_b, test_size=0.2, random_state=42)
X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(X, y_c, test_size=0.2, random_state=42)

In [13]:
# Define classification algorithms
classifiers = {
    "Logistic Regression": LogisticRegression(),
    "Naive Bayes": MultinomialNB(),
    "Random Forest": RandomForestClassifier(),
    "Support Vector Machine": SVC(),

}

In [14]:
# Function to train and evaluate models
def train_and_evaluate(X_train, X_test, y_train, y_test):
    results = {}
    for name, clf in classifiers.items():
        pipeline = Pipeline([
            ('tfidf', TfidfVectorizer()),
            ('clf', clf)
        ])
        pipeline.fit(X_train, y_train)
        y_pred = pipeline.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average='weighted')
        results[name] = {'accuracy': accuracy, 'f1_score': f1}
        print(f"{name} - Accuracy: {accuracy:.2f}, F1 Score: {f1:.2f}")
    return results


In [15]:
# Train and evaluate models for each subtask
print("Subtask A Results:")
results_a = train_and_evaluate(X_train_a, X_test_a, y_train_a, y_test_a)
print("\nSubtask B Results:")
results_b = train_and_evaluate(X_train_b, X_test_b, y_train_b, y_test_b)
print("\nSubtask C Results:")
results_c = train_and_evaluate(X_train_c, X_test_c, y_train_c, y_test_c)

Subtask A Results:
Logistic Regression - Accuracy: 0.76, F1 Score: 0.73
Naive Bayes - Accuracy: 0.70, F1 Score: 0.63
Random Forest - Accuracy: 0.75, F1 Score: 0.73
Support Vector Machine - Accuracy: 0.75, F1 Score: 0.72

Subtask B Results:


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression - Accuracy: 0.74, F1 Score: 0.70
Naive Bayes - Accuracy: 0.68, F1 Score: 0.58
Random Forest - Accuracy: 0.73, F1 Score: 0.69
Support Vector Machine - Accuracy: 0.73, F1 Score: 0.68

Subtask C Results:


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression - Accuracy: 0.73, F1 Score: 0.66
Naive Bayes - Accuracy: 0.70, F1 Score: 0.58
Random Forest - Accuracy: 0.73, F1 Score: 0.67
Support Vector Machine - Accuracy: 0.72, F1 Score: 0.64


In [32]:
#Code to predict the offensive language. (SUB Task A, B ,C covered)

def detect_offensive_language(text):
  # Preprocess the text
  # Convert the text into a Pandas Series for preprocessing
  text_series = pd.Series([text])
  clean_text = TextPreprocessor().fit_transform(text_series)[0] # Pass the Series to fit_transform

  # Classify the text for subtask A
  pipeline_a = Pipeline([
      ('tfidf', TfidfVectorizer()),
      ('clf', LogisticRegression())
  ])
  pipeline_a.fit(X_train_a, y_train_a)
  subtask_a_prediction = pipeline_a.predict([clean_text])[0]

  # If the text is not offensive, return None
  if subtask_a_prediction == 'NOT':
    return None

  # Classify the text for subtask B
  pipeline_b = Pipeline([
      ('tfidf', TfidfVectorizer()),
      ('clf', MultinomialNB())
  ])
  pipeline_b.fit(X_train_b, y_train_b)
  subtask_b_prediction = pipeline_b.predict([clean_text])[0]

  # Classify the text for subtask C
  pipeline_c = Pipeline([
      ('tfidf', TfidfVectorizer()),
      ('clf', RandomForestClassifier())
  ])
  pipeline_c.fit(X_train_c, y_train_c)
  subtask_c_prediction = pipeline_c.predict([clean_text])[0]

  # Return the results
  return {
      'offensive': True,
      'insult_type': subtask_b_prediction,
      'target': subtask_c_prediction
  }

# Example usage
text = input("Enter a text: ")
result = detect_offensive_language(text)

# Check if result is not None before accessing keys
if result is not None:
  if result['offensive']:
    print(f"The text is offensive.")
    print(f"Insult type: {result['insult_type']}")
    print(f"Target: {result['target']}")
  else:
    print("The text is not offensive.")
else:
    print("The text is not offensive.") # Handle the case where result is None

Enter a text: @USER ur so straight forword man👌 i saw u in dance dewwane and ur just talk free ky ap kitnay porrany ho industry mein and i really like ur this
The text is not offensive.
